In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam, Adamax
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, BatchNormalization, LSTM, Discretization, Dropout,TimeDistributed, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer
import tensorflow as tf
from tensorflow.keras import regularizers

In [2]:
carregando_clear = np.load('/content/drive/MyDrive/CLEAR(1).npy', mmap_mode='r')
dados_clear = carregando_clear[:5000]

carregamento_lte1m = np.load('/content/drive/MyDrive/LTE_1M.npy', mmap_mode='r')
dados_lte1m = carregamento_lte1m[:5000]

dados_wifi = np.load('/content/drive/MyDrive/WIFI.npy', mmap_mode='r')
dados_wifi = carregamento_lte1m[:5000]

In [ ]:
print(dados_clear.shape,dados_lte1m.shape,dados_wifi.shape)

(5000, 1) (5000, 1) (5000, 1)


In [ ]:


  def separate_complex_column(data):
  # Separar a parte real e imaginária em duas colunas
      real_part = data.real
      imag_part = data.imag
      return real_part,imag_part

  def montando_datasets(dados_1, dados_2):
  # juntando os dados
      real1, imag1 = separate_complex_column(dados_1)
      real2, imag2 = separate_complex_column(dados_2)
      atributos1 = np.ones(len(dados_1))
      atributos2 = np.zeros(len(dados_2))
      dados1 = np.column_stack((real1, imag1, atributos1))
      dados2 = np.column_stack((real2, imag2, atributos2))
      conjunto = np.vstack([dados1, dados2])
      indices = np.random.permutation(len(conjunto))
      data = conjunto[indices]
      return data

  def remodelar(data,feture):
  # remodelando os dados
      df = pd.DataFrame(data, columns =  ['real','imag', 'clfq'] )
      df_junt = df[['real', 'imag']].to_numpy()
      data_dim = df_junt.reshape(len(data)//100, 100, feture)
      data_dim = data_dim.astype(np.float32)
      alvo = df['clfq'].values
      alvo_dim = alvo.reshape(len(alvo)//100,100, feture-1)
      alvo_dim = alvo_dim.astype(np.float32)
      return data_dim,alvo_dim


def processos(dados_1,dados_2,feture): 
    data = montando_datasets(dados_1, dados_2)
    X , Y = remodelar(data,feture)
    return X, Y

In [ ]:
X,Y = processos(dados_clear, dados_lte1m,2)
print(X)

[[[-3.0518622e-05  0.0000000e+00]
  [ 3.0518622e-05  1.5259311e-04]
  [ 9.1555863e-05  2.7466760e-04]
  ...
  [ 2.1363035e-04  3.0518622e-05]
  [-2.1363035e-04 -1.5259311e-04]
  [ 2.4414898e-04 -1.2207449e-04]]

 [[-9.1555863e-05  6.1037244e-05]
  [ 0.0000000e+00  6.1037244e-05]
  [ 3.0518622e-05 -6.1037244e-05]
  ...
  [ 2.1363035e-04  3.0518622e-05]
  [ 2.7466760e-04  3.0518622e-05]
  [ 0.0000000e+00 -1.8311173e-04]]

 [[ 9.1555863e-05  1.5259311e-04]
  [ 1.5259311e-04 -6.1037244e-05]
  [-3.0518622e-05 -6.1037244e-05]
  ...
  [ 9.1555863e-05 -1.5259311e-04]
  [-1.8311173e-04  3.0518622e-05]
  [-1.5259311e-04  2.1363035e-04]]

 ...

 [[ 2.1363035e-04  2.1363035e-04]
  [ 3.0518622e-05  3.0518622e-05]
  [-9.1555863e-05  0.0000000e+00]
  ...
  [ 3.0518622e-05 -9.1555863e-05]
  [-2.7466760e-04 -1.5259311e-04]
  [-1.2207449e-04 -3.0518622e-05]]

 [[-9.1555863e-05 -9.1555863e-05]
  [ 0.0000000e+00  2.7466760e-04]
  [-1.5259311e-04 -1.5259311e-04]
  ...
  [ 3.0518622e-05  1.8311173e-04]
  [ 

# Clear x Lte1m

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=None, train_size=0.7,random_state=42)

model = Sequential()
model.add(BatchNormalization(input_shape=(None, 2)))
model.add(SimpleRNN(units=528,input_shape=(None, 2),activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(SimpleRNN(units=264,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(SimpleRNN(units=128, activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(SimpleRNN(units=64,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(Dense(units=32,activation='tanh'))
model.add(Dense(units=16,activation="relu"))
model.add(Dense(units=8, activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))


opt = Adamax(
    learning_rate=0.0003,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)

model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy','AUC', 'Precision', 'Recall'])
model.fit(X_train, y_train, epochs=100, batch_size=32)
model.summary()

Epoch 1/100
3/3 [==============================] - 7s 363ms/step - loss: 0.6940 - accuracy: 0.4916 - auc: 0.4938 - precision: 0.4778 - recall: 0.1409
Epoch 2/100
3/3 [==============================] - 1s 384ms/step - loss: 0.6937 - accuracy: 0.5040 - auc: 0.4944 - precision: 0.5070 - recall: 0.4204
Epoch 3/100
3/3 [==============================] - 1s 365ms/step - loss: 0.6940 - accuracy: 0.5004 - auc: 0.5045 - precision: 0.5046 - recall: 0.2474
Epoch 4/100
3/3 [==============================] - 2s 558ms/step - loss: 0.6936 - accuracy: 0.5029 - auc: 0.5071 - precision: 0.5032 - recall: 0.7492
Epoch 5/100
3/3 [==============================] - 2s 629ms/step - loss: 0.6934 - accuracy: 0.4993 - auc: 0.4968 - precision: 0.5010 - recall: 0.5716
Epoch 6/100
3/3 [==============================] - 2s 477ms/step - loss: 0.6929 - accuracy: 0.5047 - auc: 0.5141 - precision: 0.5040 - recall: 0.8235
Epoch 7/100
3/3 [==============================] - 1s 374ms/step - loss: 0.6930 - accuracy: 0.5086 -

In [ ]:
trainPredict = model.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())

1/1 [==============================] - 1s 696ms/step


In [ ]:
print(cm)

[[570 958]
 [578 894]]


# Clear x Wifi

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape=(None, 2)))
model.add(SimpleRNN(units=528,input_shape=(None, 2),activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(SimpleRNN(units=264,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(SimpleRNN(units=128, activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(SimpleRNN(units=64,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(Dense(units=32,activation='tanh'))
model.add(Dense(units=16,activation="relu"))
model.add(Dense(units=8, activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))


opt = Adamax(
    learning_rate=0.0003,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)
X2,Y2 = processos(dados_clear, dados_wifi,2)
X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=None, train_size=0.7,random_state=42)
model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy','AUC', 'Precision', 'Recall'])
model.fit(X_train, y_train, epochs=100, batch_size=16)
model.summary()

Epoch 1/100
5/5 [==============================] - 53s 424ms/step - loss: 0.6942 - accuracy: 0.5056 - auc: 0.5096 - precision: 0.5079 - recall: 0.6083
Epoch 2/100
5/5 [==============================] - 2s 315ms/step - loss: 0.6957 - accuracy: 0.5014 - auc: 0.4961 - precision: 0.5046 - recall: 0.5859
Epoch 3/100
5/5 [==============================] - 2s 322ms/step - loss: 0.6951 - accuracy: 0.4994 - auc: 0.4946 - precision: 0.5032 - recall: 0.5272
Epoch 4/100
5/5 [==============================] - 2s 325ms/step - loss: 0.6944 - accuracy: 0.4983 - auc: 0.4988 - precision: 0.5019 - recall: 0.5876
Epoch 5/100
5/5 [==============================] - 2s 327ms/step - loss: 0.6950 - accuracy: 0.5033 - auc: 0.5056 - precision: 0.5048 - recall: 0.7611
Epoch 6/100
5/5 [==============================] - 2s 323ms/step - loss: 0.6930 - accuracy: 0.5166 - auc: 0.5175 - precision: 0.5166 - recall: 0.6352
Epoch 7/100
5/5 [==============================] - 2s 383ms/step - loss: 0.6933 - accuracy: 0.5049 

In [ ]:
trainPredict = model.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())
print(cm)

1/1 [==============================] - 1s 695ms/step
[[763 767]
 [730 740]]


# Lte1m x wifi

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape=(None, 2)))
model.add(SimpleRNN(units=528,input_shape=(None, 2),activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(SimpleRNN(units=264,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(SimpleRNN(units=128, activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(SimpleRNN(units=64,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(Dense(units=32,activation='tanh'))
model.add(Dense(units=16,activation="relu"))
model.add(Dense(units=8, activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))


opt = Adamax(
    learning_rate=0.0003,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)

X3,Y3 = processos(dados_lte1m, dados_wifi,2)
X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=None, train_size=0.7,random_state=42)
model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy','AUC', 'Precision', 'Recall'])
model.fit(X_train, y_train, epochs=100, batch_size=16)
model.summary()

Epoch 1/100
5/5 [==============================] - 9s 329ms/step - loss: 0.6938 - accuracy: 0.5036 - auc: 0.4977 - precision: 0.5038 - recall: 0.9972
Epoch 2/100
5/5 [==============================] - 2s 315ms/step - loss: 0.6939 - accuracy: 0.5061 - auc: 0.5089 - precision: 0.5061 - recall: 0.8362
Epoch 3/100
5/5 [==============================] - 2s 328ms/step - loss: 0.6931 - accuracy: 0.5086 - auc: 0.5108 - precision: 0.5084 - recall: 0.7543
Epoch 4/100
5/5 [==============================] - 2s 326ms/step - loss: 0.6929 - accuracy: 0.5126 - auc: 0.5186 - precision: 0.5159 - recall: 0.5349
Epoch 5/100
5/5 [==============================] - 2s 321ms/step - loss: 0.6925 - accuracy: 0.5170 - auc: 0.5162 - precision: 0.5205 - recall: 0.5295
Epoch 6/100
5/5 [==============================] - 2s 370ms/step - loss: 0.6925 - accuracy: 0.5127 - auc: 0.5261 - precision: 0.5224 - recall: 0.3866
Epoch 7/100
5/5 [==============================] - 3s 533ms/step - loss: 0.6916 - accuracy: 0.5243 -

In [ ]:
trainPredict = model.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())
print(cm)

1/1 [==============================] - 0s 139ms/step
[[791 737]
 [755 717]]


# Teste de Dropout

In [ ]:
X,Y = processos(dados_clear, dados_lte1m,2)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=None, train_size=0.7,random_state=42)

model = Sequential()
model.add(BatchNormalization(input_shape=(None, 2)))
model.add(SimpleRNN(units=528,input_shape=(None, 2),activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal",return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=264,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=128, activation="tanh",kernel_initializer="glorot_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=64,activation="relu",kernel_initializer="he_uniform",recurrent_initializer="orthogonal", return_sequences=True))
model.add(Dropout(0.2))

model.add(Dense(units=32,activation='tanh'))
model.add(Dense(units=16,activation="relu"))
model.add(Dense(units=8, activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))


opt = Adamax(
    learning_rate=0.0003,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)

model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy','AUC', 'Precision', 'Recall'])
model.fit(X_train, y_train, epochs=300, batch_size=32)
model.summary()

Epoch 1/300
11/11 [==============================] - 13s 738ms/step - loss: 0.6949 - accuracy: 0.4977 - auc: 0.4999 - precision: 0.4966 - recall: 0.5514
Epoch 2/300
11/11 [==============================] - 6s 563ms/step - loss: 0.6940 - accuracy: 0.5051 - auc: 0.5040 - precision: 0.5052 - recall: 0.3503
Epoch 3/300
11/11 [==============================] - 8s 765ms/step - loss: 0.6940 - accuracy: 0.5008 - auc: 0.4966 - precision: 0.4986 - recall: 0.2608
Epoch 4/300
11/11 [==============================] - 6s 558ms/step - loss: 0.6936 - accuracy: 0.5029 - auc: 0.5031 - precision: 0.5024 - recall: 0.2832
Epoch 5/300
11/11 [==============================] - 8s 730ms/step - loss: 0.6937 - accuracy: 0.5039 - auc: 0.5012 - precision: 0.5035 - recall: 0.3405
Epoch 6/300
11/11 [==============================] - 6s 548ms/step - loss: 0.6933 - accuracy: 0.5025 - auc: 0.5035 - precision: 0.5022 - recall: 0.2304
Epoch 7/300
11/11 [==============================] - 8s 752ms/step - loss: 0.6932 - acc

In [ ]:
trainPredict = model.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())
print(cm)

5/5 [==============================] - 1s 128ms/step
[[4239 3208]
 [4103 3450]]


# Aplicando fourier

In [9]:
def fourier_processing (sinal_1, sinal_2):
    fft_1 = np.fft.fft(sinal_1)
    fft_2 = np.fft.fft(sinal_2)
    espectro_1 = np.abs(fft_1)
    espectro_2 = np.abs(fft_2)
    frequencias_1 = np.fft.fftfreq(len(sinal_1))
    frequencias_2 = np.fft.fftfreq(len(sinal_2))
    atributos1 = np.ones(len(sinal_1))
    atributos2 = np.zeros(len(sinal_2))
    dados1 = np.column_stack((espectro_1, frequencias_1, atributos1))
    dados2 = np.column_stack((espectro_2, frequencias_2, atributos2))
    conjunto = np.vstack([dados1, dados2])
    indices = np.random.permutation(len(conjunto))
    data = conjunto[indices]
    return data


def remodelar_fourier(data,feture):
    df = pd.DataFrame(data, columns =  ['ampli','freq', 'clfq'] )
    df_junt = df[['ampli', 'freq']].to_numpy()
    data_dim = df_junt.reshape(len(data)//100, 100, feture)
    data_dim = data_dim.astype(np.float32)
    alvo = df['clfq'].values
    alvo_dim = alvo.reshape(len(alvo)//100,100, feture-1)
    alvo_dim = alvo_dim.astype(np.float32)
    return data_dim,alvo_dim

def processing_fourier(dados_1,dados_2,feture): 
    data = fourier_processing(dados_1, dados_2)
    X_f , Y_f = remodelar_fourier(data,feture)
    return X_f, Y_f

In [ ]:
X_f, Y_f = processing_fourier(dados_clear,dados_lte1m,2)

In [ ]:
print(X_f)

[[[ 3.0670836e-04  2.8400000e-02]
  [ 1.9541453e-04  2.4580000e-01]
  [ 1.5259311e-04  3.2120001e-01]
  ...
  [ 6.8241716e-05 -4.0020001e-01]
  [ 2.1363035e-04  1.2220000e-01]
  [ 1.9541453e-04 -2.5299999e-01]]

 [[ 3.0518622e-05  1.3300000e-01]
  [ 1.5259311e-04  4.1280001e-01]
  [ 1.7263940e-04 -1.1220000e-01]
  ...
  [ 2.2217892e-04  2.6940000e-01]
  [ 2.1579926e-04  4.7799999e-01]
  [ 3.5590524e-04  3.3599999e-02]]

 [[ 1.2583150e-04  1.1140000e-01]
  [ 4.2834927e-04  4.8559999e-01]
  [ 2.8136780e-04  4.4880000e-01]
  ...
  [ 9.6508353e-05  3.9600000e-02]
  [ 4.3159848e-04  1.6940001e-01]
  [ 2.3835806e-04  3.3520001e-01]]

 ...

 [[ 3.9082905e-04  4.6599999e-01]
  [ 3.0518623e-04  2.5619999e-01]
  [ 2.0472513e-04  4.5519999e-01]
  ...
  [ 2.0472513e-04 -2.7039999e-01]
  [ 1.2947954e-04 -9.8600000e-02]
  [ 1.9541453e-04 -3.4540001e-01]]

 [[ 1.1003645e-04 -6.9200002e-02]
  [ 2.4604899e-04  1.2639999e-01]
  [ 1.9541453e-04  3.6440000e-01]
  ...
  [ 1.3648343e-04 -1.4139999e-01]
  [ 

In [19]:
def model_simpleRNN(dados_1,dados_2):
    X_f,Y_f = processing_fourier(dados_1, dados_2,2)

    X_train, X_test, y_train, y_test = train_test_split(X_f, Y_f, test_size=0.3, train_size=0.7,random_state=42)

    model = Sequential()
    model.add(BatchNormalization(input_shape=(None, 2)))

    model.add(SimpleRNN(units=528,
                        input_shape=(None, 2),
                        activation="tanh",
                        kernel_initializer="glorot_uniform",
                        recurrent_initializer="orthogonal",
                        dropout=0.1,recurrent_dropout=0.1,
                        return_sequences=True))

    model.add(SimpleRNN(units=256,
                        activation="relu",
                        kernel_initializer="he_uniform",
                        recurrent_initializer="orthogonal", 
                        dropout=0.1,recurrent_dropout=0.1,
                        return_sequences=True))

    model.add(SimpleRNN(units=128, 
                        activation="tanh",
                        kernel_initializer="glorot_uniform",
                        recurrent_initializer="orthogonal",
                        dropout=0.1,recurrent_dropout=0.1, 
                        return_sequences=True))

    model.add(SimpleRNN(units=64,
                        activation="relu",
                        kernel_initializer="he_uniform",
                        recurrent_initializer="orthogonal",
                        dropout=0.1, recurrent_dropout=0.0, 
                        return_sequences=True))


    model.add(Dense(units=32,activation='tanh'))
    model.add(Dense(units=16,activation="relu"))
    model.add(Dense(units=8, activation='tanh'))
    model.add(Dense(units=1, activation='sigmoid'))


    opt = Adamax(
        learning_rate=0.0003,
        beta_1=0.9,
        beta_2=0.99,
        epsilon=1e-07)

    mc = model.compile(loss='binary_crossentropy', 
                       optimizer =opt, 
                       metrics=['accuracy','AUC', 'Precision', 'Recall'])
    
    mf = model.fit(X_train, y_train, 
                   epochs=100, 
                   batch_size=32)
    
    ms = model.summary()
    return model,X_test,y_test

clear x lte1m

In [20]:
model,X_test,y_test = model_simpleRNN(dados_clear,dados_lte1m)

Epoch 1/100
3/3 [==============================] - 8s 651ms/step - loss: 0.7019 - accuracy: 0.4943 - auc: 0.4977 - precision: 0.4926 - recall: 0.5710
Epoch 2/100
3/3 [==============================] - 3s 873ms/step - loss: 0.6984 - accuracy: 0.5110 - auc: 0.5090 - precision: 0.5090 - recall: 0.4649
Epoch 3/100
3/3 [==============================] - 2s 518ms/step - loss: 0.6982 - accuracy: 0.5084 - auc: 0.5023 - precision: 0.5060 - recall: 0.4744
Epoch 4/100
3/3 [==============================] - 2s 507ms/step - loss: 0.6970 - accuracy: 0.5079 - auc: 0.5091 - precision: 0.5052 - recall: 0.4859
Epoch 5/100
3/3 [==============================] - 2s 517ms/step - loss: 0.6964 - accuracy: 0.5071 - auc: 0.5080 - precision: 0.5043 - recall: 0.5011
Epoch 6/100
3/3 [==============================] - 2s 506ms/step - loss: 0.6956 - accuracy: 0.5121 - auc: 0.5111 - precision: 0.5089 - recall: 0.5333
Epoch 7/100
3/3 [==============================] - 2s 741ms/step - loss: 0.6969 - accuracy: 0.4949 -

In [21]:
trainPredict = model.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())
print(cm)

1/1 [==============================] - 1s 827ms/step
[[711 769]
 [748 772]]


clear x wifi

In [22]:
model_2,X_test,y_test = model_simpleRNN(dados_clear,dados_wifi)

Epoch 1/100
3/3 [==============================] - 9s 642ms/step - loss: 0.7169 - accuracy: 0.5023 - auc: 0.5073 - precision: 0.4884 - recall: 0.0604
Epoch 2/100
3/3 [==============================] - 4s 1s/step - loss: 0.7000 - accuracy: 0.4983 - auc: 0.4999 - precision: 0.4889 - recall: 0.2409
Epoch 3/100
3/3 [==============================] - 3s 867ms/step - loss: 0.6959 - accuracy: 0.5044 - auc: 0.5055 - precision: 0.5008 - recall: 0.4439
Epoch 4/100
3/3 [==============================] - 2s 518ms/step - loss: 0.6958 - accuracy: 0.5066 - auc: 0.5026 - precision: 0.5025 - recall: 0.5786
Epoch 5/100
3/3 [==============================] - 2s 605ms/step - loss: 0.6947 - accuracy: 0.5003 - auc: 0.5032 - precision: 0.4973 - recall: 0.6344
Epoch 6/100
3/3 [==============================] - 3s 649ms/step - loss: 0.6939 - accuracy: 0.5079 - auc: 0.5083 - precision: 0.5031 - recall: 0.6808
Epoch 7/100
3/3 [==============================] - 2s 551ms/step - loss: 0.6939 - accuracy: 0.5090 - au

In [23]:
trainPredict = model_2.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())
print(cm)

1/1 [==============================] - 1s 727ms/step
[[726 748]
 [757 769]]


wifi x lte1m

In [24]:
model_3,X_test,y_test = model_simpleRNN(dados_lte1m,dados_wifi)

Epoch 1/100
3/3 [==============================] - 8s 481ms/step - loss: 0.7153 - accuracy: 0.4977 - auc: 0.4965 - precision: 0.4928 - recall: 0.5215
Epoch 2/100
3/3 [==============================] - 2s 490ms/step - loss: 0.7094 - accuracy: 0.5076 - auc: 0.5043 - precision: 0.5024 - recall: 0.4900
Epoch 3/100
3/3 [==============================] - 2s 482ms/step - loss: 0.7062 - accuracy: 0.5087 - auc: 0.5067 - precision: 0.5037 - recall: 0.4736
Epoch 4/100
3/3 [==============================] - 2s 486ms/step - loss: 0.7036 - accuracy: 0.5097 - auc: 0.5088 - precision: 0.5048 - recall: 0.4684
Epoch 5/100
3/3 [==============================] - 2s 488ms/step - loss: 0.7026 - accuracy: 0.5003 - auc: 0.5049 - precision: 0.4947 - recall: 0.4721
Epoch 6/100
3/3 [==============================] - 2s 678ms/step - loss: 0.7026 - accuracy: 0.4999 - auc: 0.5008 - precision: 0.4945 - recall: 0.4964
Epoch 7/100
3/3 [==============================] - 3s 811ms/step - loss: 0.6997 - accuracy: 0.5126 -

In [25]:
trainPredict = model_2.predict(X_test)
trainPredict_binary = (trainPredict > 0.5).astype(int)

cm = confusion_matrix(y_test.flatten(), trainPredict_binary.flatten())
print(cm)

1/1 [==============================] - 0s 413ms/step
[[717 746]
 [762 775]]
